In [1]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt

In [50]:
def fetch_stock_data(ticker, api_key):
    url = f'https://yfapi.net/v8/finance/chart/{ticker}'
    headers = {'X-API-KEY': api_key }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # raises an error when not 200
        # parse the json
        data = response.json()
        if data['chart']['result'] == None:
          print("Error trying to read stock")
          return None
        print("Stock data fetched successfully!")
        return data

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"Error occurred: {err}")
    except KeyError:
        print("Data format has changed or stock data unavailable.")
    return None


In [55]:
# Get ticker symbol and API key from the user
ticker = input('Enter stock ticker symbol: ').upper()
api_key = ""

# Display ticker symbol
print('The ticker symbol is: ' + ticker)

Enter stock ticker symbol: AAPL
The ticker symbol is: AAPL


In [62]:

data = fetch_stock_data(ticker, api_key)

# display stock info
if data:
    try:
        stock_info = {
            'Ticker Name': data['chart']['result'][0]["meta"]["symbol"],
            'Full Name of Stock': data['chart']['result'][0]["meta"]["longName"],
            'Current Market Price': data['chart']['result'][0]["meta"]["regularMarketPrice"],
            '52 Week High': data['chart']['result'][0]["meta"]["fiftyTwoWeekHigh"],
            '52 Week Low': data['chart']['result'][0]["meta"]["fiftyTwoWeekLow"]
        }

        #  stock information
        print("\nStock Information:")
        for key, value in stock_info.items():
            print(f"{key}: {value}")
    except KeyError:
        print("Failed to extract stock information.")


Stock data fetched successfully!

Stock Information:
Ticker Name: AAPL
Full Name of Stock: Apple Inc.
Current Market Price: 227.725
52 Week High: 229.52
52 Week Low: 227.3


In [59]:
def fetch_trending_stocks(api_key):
    trending_url = 'https://yfapi.net/v1/finance/trending/US'
    headers = {'X-API-KEY': api_key}

    try:
        trending_response = requests.get(trending_url, headers=headers)
        trending_response.raise_for_status()
        trending_data = trending_response.json()
        # if trending_data['chart']['result'] == None:
        #   print("Error trying to read stock")
        #   return None
        print("Trending stocks data fetched successfully!")
        return trending_data

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"Error occurred: {err}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return None


In [60]:
trending_data = fetch_trending_stocks(api_key)

if trending_data and 'finance' in trending_data and 'result' in trending_data['finance']:
    trending_stocks = trending_data['finance']['result'][0]['quotes'][:5]  # Top 5 trending stocks
    print("\nTrending Stocks:")
    trending_stocks_info = []
    for stock in trending_stocks:
        trending_info = {'Ticker Name': stock['symbol']}
        trending_stocks_info.append(trending_info)
        print(f"{stock['symbol']}")
else:
    print("No trending stocks found or the response format is not as expected.")


Trending stocks data fetched successfully!

Trending Stocks:
NQ=F
ES=F
YM=F
AMSC
GC=F


In [61]:
# Combine stock information and trending stocks into a DataFrame
if data and trending_stocks_info:
    combined_data = trending_stocks_info + [stock_info]  # Add stock info to trending stocks
    combined_df = pd.DataFrame(combined_data)

    # Save to CSV
    combined_df.to_csv(f'{ticker}_stock_trending_info.csv', index=False)
    print(f"\nData has been saved to {ticker}_stock_trending_info.csv")
else:
    print("No data to save.")



Data has been saved to AAPL_stock_trending_info.csv
